In [2]:
import time
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

option = webdriver.ChromeOptions()
option.add_argument("--incognito")


driver = webdriver.Chrome("D:/chromedriver_win32/chromedriver.exe",chrome_options=option)
# driver.get('https://twitter.com/maggiew61/following')
driver.get('https://twitter.com/?logged_out=1&lang=en')
wait = WebDriverWait(driver, 10)  #wait until element visible

user = wait.until(EC.visibility_of_element_located((By.NAME, "session[username_or_email]")))
user.clear()
user.send_keys('maggiew61@yahoo.com')


pass1 = wait.until(EC.visibility_of_element_located((By.NAME, "session[password]")))
pass1.clear()
pass1.send_keys('nnlily2002')


login = driver.find_element_by_class_name('EdgeButton')
login.submit()
# login.click()
sleep(2)




#### GO TO FOLLOWIN PAGE




In [3]:
from bs4 import BeautifulSoup as bs
#click on following 
myName = 'maggiew61'
driver.base_url = "https://twitter.com/" + myName + '/following?lang=en'
driver.get(driver.base_url)
# print('driver.base_url is',driver.base_url)
# html_source = driver.page_source
# dailyemail_links = html_source.encode('utf-8')
# soup=bs(dailyemail_links, "lxml")

#### INFINITE SCROLL

In [4]:
for i in range(1,4):  #execute sth a number of times,starting from 1 and end at 7
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    print(i)

1
2
3


#### GET DATA 

In [9]:
from bs4 import BeautifulSoup as bs
driver.execute_script("window.scrollTo(0, 0);") #scroll to the top of page otherwise can't get the <a>elements
soup = BeautifulSoup(driver.page_source ,"html.parser")
arr = [x.div['data-screen-name'] for x in soup.body.find_all('div', attrs={'data-item-type':'user'})]
bios = [x.p.text for x in soup.body.findAll('div', attrs={'data-item-type':'user'})]
fullnames = [x.text.strip() for x in soup.body.findAll('a', 'fullname')][1:] # avoid your own name
def getTimestamps(x):
    temp = x.findAll('span', '_timestamp')
    if len(temp) > 0:
        return temp[0].get('data-time')
    else:
        return None
    
# now get the user's own timeline
for i in range(0,len(arr)):
    currentUser = arr[i]
    print('doing user:' + str(i) + ' ' + currentUser)
    driver.find_element_by_xpath("//a[contains(.,'" + currentUser + "')]").click()
    html_source = driver.page_source
    dailyemail_links = html_source.encode('utf-8')
    soup=bs(dailyemail_links, "lxml")
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    # name
    name = soup.find('a', "ProfileHeaderCard-nameLink").text
    # loc
    temp = soup.find('span', 'ProfileHeaderCard-locationText')
    temp = temp.text if temp else ''
    loc = temp.strip() if temp else ''
    # url
    temp = soup.find('span', 'ProfileHeaderCard-urlText')
    temp = temp.a if temp else None
    temp2 = temp.get('title') if temp else None
    url = temp2 if temp2 else (temp.get('href') if temp else None)
    # stats
    temp = soup.find('a',{'data-nav': 'tweets'})
    stats_tweets = temp.find('span', 'ProfileNav-value')['data-count'] if temp else 0
    temp = soup.find('a',{'data-nav': 'following'})
    stats_following = temp.find('span', 'ProfileNav-value')['data-count'] if temp else 0
    temp = soup.find('a',{'data-nav': 'followers'})
    stats_followers = temp.find('span', 'ProfileNav-value')['data-count'] if temp else 0
    temp = soup.find('a',{'data-nav': 'favorites'})
    stats_favorites = temp.find('span', 'ProfileNav-value')['data-count'] if temp else 0
    # all text
    text = [''.join(x.findAll(text=True)) for x in soup.body.findAll('p', 'tweet-text')]
    # most recent activity
    alltweets = soup.body.findAll('li', attrs={'data-item-type':'tweet'})
    tweetTimestamps = list(map(getTimestamps, alltweets)) if len(alltweets) > 0 else 0
    # engagements
    noretweets = [x.findAll('span', 'ProfileTweet-actionCount') for x in alltweets if not x.div.get('data-retweet-id')]
    templist = [x.findAll('span', 'ProfileTweet-actionCount') for x in alltweets if not x.div.get('data-retweet-id')]
    templist = [item for sublist in templist for item in sublist]
    engagements = sum([int(x.get('data-tweet-stat-count')) for x in templist if x.get('data-tweet-stat-count')])
    
    #TEST
    weatherTable = dict(
    user= [currentUser],
    text= [text],
    mostrecentTimestamp= [tweetTimestamps],
    engagements= [engagements],
    name= [name],
    loc= [loc],
    url= [url],
    stats_tweets= [stats_tweets],
    stats_following= [stats_following],
    stats_followers= [stats_followers],
    stats_favorites=[stats_favorites],
        )
    print('weatherTable is',weatherTable)

    #TEST ENDS
#     main = pd.concat([main, pd.DataFrame(data = {
#         'user': [currentUser],
#         'text': [text],
#         'mostrecentTimestamp': [tweetTimestamps],
#         'engagements': [engagements],
#         'name': [name],
#         'loc': [loc],
#         'url': [url],
#         'stats_tweets': [stats_tweets],
#         'stats_following': [stats_following],
#         'stats_followers': [stats_followers],
#         'stats_favorites': [stats_favorites],
#     })])
    driver.back()
    sleep(5)

doing user:0 ArianaGrande


TypeError: 'str' object is not callable

#### SINGLE DATA

In [42]:
from bs4 import BeautifulSoup
driver.execute_script("window.scrollTo(0, 0);") #scroll to the top of page otherwise can't get the <a>elements
soup = BeautifulSoup(driver.page_source ,"html.parser")
arr = [x.div['data-screen-name'] for x in soup.body.find_all('div', attrs={'data-item-type':'user'})]
bios = [x.p.text for x in soup.body.findAll('div', attrs={'data-item-type':'user'})]
fullnames = [x.text.strip() for x in soup.body.findAll('a', 'fullname')][1:] # avoid your own name
def getTimestamps(x):
    temp = x.findAll('span', '_timestamp')
    if len(temp) > 0:
        return temp[0].get('data-time')
    else:
        return None
    
# now get the user's own timeline
# for i in range(0,len(arr)):
currentUser = arr[2]
print('doing user:' + str(i) + ' ' + currentUser)
driver.find_element_by_xpath("//a[contains(.,'" + currentUser + "')]").click()

soup = BeautifulSoup(driver.page_source ,"html.parser")  #每到一個新頁面都要放這個
html_source = driver.page_source
dailyemail_links = html_source.encode('utf-8')
soup=bs(dailyemail_links, "lxml")
time.sleep(2)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(1)
# name
name = soup.find('a', "ProfileHeaderCard-nameLink").text

# locaction
temp = soup.find('span', 'ProfileHeaderCard-locationText')
temp = temp.text if temp else ''
loc = temp.strip() if temp else ''

# url in bio
temp = soup.find('span', 'ProfileHeaderCard-urlText')
temp = temp.a if temp else None
temp2 = temp.get('title') if temp else None
url = temp2 if temp2 else (temp.get('href') if temp else None)

# stats(tweets/following/followers/likes)
temp = soup.find('a',{'data-nav': 'tweets'})
stats_tweets = temp.find('span', 'ProfileNav-value')['data-count'] if temp else 0
temp = soup.find('a',{'data-nav': 'following'})
stats_following = temp.find('span', 'ProfileNav-value')['data-count'] if temp else 0
temp = soup.find('a',{'data-nav': 'followers'})
stats_followers = temp.find('span', 'ProfileNav-value')['data-count'] if temp else 0
temp = soup.find('a',{'data-nav': 'favorites'})
stats_favorites = temp.find('span', 'ProfileNav-value')['data-count'] if temp else 0

# all text
# text = [''.join(x.find_all(text=True)) for x in soup.body.find_all('p', 'tweet-text')]

#test
each_row = soup.find_all('div',{'class': 'js-stream-tweet'}) #collect all the rows in a list


i = 0
collect = []
for i in range(0,len(each_row)):  #each tweet
#     text = [''.join(x.find_all(text=True)) for x in soup.body.find_all('p', 'tweet-text')]

    temp = each_row[i].find_all('span',{'class': 'ProfileTweet-actionCount'}) #all tags in the bottom feed
    socialFeed = dict(
    msg = temp[0]['data-tweet-stat-count'],
    retweet = temp[1]['data-tweet-stat-count'],
    like = temp[2]['data-tweet-stat-count']
    )


    test1234 = each_row[i].find_all('p', 'tweet-text')
    test12345 = test1234[0].find_all(text=True)
    print('test12345  is',test12345,'socialFeed is',socialFeed)
    

#     collect.append(test123)
# print('collect is',collect)
#test

# most recent activity
alltweets = soup.body.findAll('li', attrs={'data-item-type':'tweet'})
tweetTimestamps = list(map(getTimestamps, alltweets)) if len(alltweets) > 0 else 0

# engagements
noretweets = [x.findAll('span', 'ProfileTweet-actionCount') for x in alltweets if not x.div.get('data-retweet-id')]
templist = [x.findAll('span', 'ProfileTweet-actionCount') for x in alltweets if not x.div.get('data-retweet-id')]
templist = [item for sublist in templist for item in sublist]
#had prblem, come back later
# engagements = sum([int(x.get('data-tweet-stat-count')) for x in templist if x.get('data-tweet-stat-count')]) 

#put it in json
weatherTable = dict(
user= [currentUser],
text= [text],
mostrecentTimestamp= [tweetTimestamps],
# engagements= [engagements],
name= [name],
loc= [loc],
url= [url],
stats_tweets= [stats_tweets],
stats_following= [stats_following],
stats_followers= [stats_followers],
stats_favorites=[stats_favorites],
    )
print('weatherTable is',weatherTable)

#     main = pd.concat([main, pd.DataFrame(data = {
#         'user': [currentUser],
#         'text': [text],
#         'mostrecentTimestamp': [tweetTimestamps],
#         'engagements': [engagements],
#         'name': [name],
#         'loc': [loc],
#         'url': [url],
#         'stats_tweets': [stats_tweets],
#         'stats_following': [stats_following],
#         'stats_followers': [stats_followers],
#         'stats_favorites': [stats_favorites],
#     })])
driver.back()
sleep(5)

doing user:19 KrisJenner
test12345  is ['Get my good friend ', '@', 'AROD', "’s first ever bat today before it sells out! It's limited edition, only 696 available! ", '#', 'arod', ' ', '#', 'arodgs25', ' ', 'http://www.', 'maruccisports.com/gs25', '\xa0', 'pic.twitter.com/6xmGvPwluQ'] socialFeed is {'msg': '19', 'retweet': '46', 'like': '627'}
test12345  is ['Don’t miss my episode of ‘OBJECTified’ with ', '#', 'harveylevin', ' tonight at 8p ET on Fox News Channel! ', '#', 'Objectified', 'pic.twitter.com/Hq3plgjJ5g'] socialFeed is {'msg': '101', 'retweet': '131', 'like': '1623'}
test12345  is ["Don't miss my episode of 'OBJECTified,' this Sunday at 8p ET on Fox News Channel! ", '@', 'HarveyLevinTMZ', 'pic.twitter.com/79aWaEBtQh'] socialFeed is {'msg': '83', 'retweet': '162', 'like': '1807'}
test12345  is ['My beauty on the cover of ', '@', 'Forbes', '! I am SO proud of you ', '@', 'kyliejenner', ' and everything you have accomplished. You work so tirelessly on your business, and with so